# prepare

In [2]:
import graphlab
song_data = graphlab.SFrame('../../lecturenote/song_data.gl/')
graphlab.canvas.set_target('ipynb')

In [3]:
users = song_data['user_id'].unique()

In [4]:
len(users)

66346

# 1. Counting unq users

In [8]:
# Kanye West
len(song_data[song_data['artist'] == 'Kanye West']['user_id'].unique())

2522

In [9]:
# Foo Fighters
len(song_data[song_data['artist'] == 'Foo Fighters']['user_id'].unique())

2055

In [10]:
# Taylor Swift
len(song_data[song_data['artist'] == 'Taylor Swift']['user_id'].unique())

3246

In [11]:
# Lady GaGa
len(song_data[song_data['artist'] == 'Lady GaGa']['user_id'].unique())

2928

# 2. find the most popular artist using group by

In [12]:
popular_artist = song_data.groupby(
    key_columns='artist',
    operations={
        'total_count': graphlab.aggregate.SUM('listen_count')
    }
)

In [15]:
popular_artist = popular_artist.sort('total_count', ascending=False)

In [16]:
popular_artist.head()
po

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


# 3. find most recommended songs using group by

In [17]:
train_data, test_data = song_data.random_split(.8, seed=0)

In [18]:
item_simi_recommender = graphlab.item_similarity_recommender.create(
    train_data, user_id='user_id', item_id='song'
)

PROGRESS: Recsys training: model = item_similarity
PROGRESS: Warning: Ignoring columns song_id, listen_count, title, artist;
PROGRESS:     To use one of these as a target column, set target = <column_name>
PROGRESS:     and use a method that allows the use of a target.
PROGRESS: Preparing data set.
PROGRESS:     Data has 893580 observations with 66085 users and 9952 items.
PROGRESS:     Data prepared in: 0.71078s
PROGRESS: Computing item similarity statistics:
PROGRESS: Computing most similar items for 9952 items:
PROGRESS: +-----------------+-----------------+
PROGRESS: | Number of items | Elapsed Time    |
PROGRESS: +-----------------+-----------------+
PROGRESS: | 1000            | 1.40111         |
PROGRESS: | 2000            | 1.43643         |
PROGRESS: | 3000            | 1.47014         |
PROGRESS: | 4000            | 1.50326         |
PROGRESS: | 5000            | 1.53599         |
PROGRESS: | 6000            | 1.56821         |
PROGRESS: | 7000            | 1.60222         |


In [19]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [22]:
subset_recommended_songs = \
    item_simi_recommender.recommend(subset_test_users,k=1)

PROGRESS: recommendations finished on 1000/10000 queries. users per second: 1677.39
PROGRESS: recommendations finished on 2000/10000 queries. users per second: 1710.17
PROGRESS: recommendations finished on 3000/10000 queries. users per second: 1724.72
PROGRESS: recommendations finished on 4000/10000 queries. users per second: 1714.41
PROGRESS: recommendations finished on 5000/10000 queries. users per second: 1712.65
PROGRESS: recommendations finished on 6000/10000 queries. users per second: 1718.69
PROGRESS: recommendations finished on 7000/10000 queries. users per second: 1719.74
PROGRESS: recommendations finished on 8000/10000 queries. users per second: 1726.19
PROGRESS: recommendations finished on 9000/10000 queries. users per second: 1725.13
PROGRESS: recommendations finished on 10000/10000 queries. users per second: 1725.47


In [23]:
most_recommended_songs = \
    subset_recommended_songs.groupby(
        key_columns='song',
        operations={
            'count': graphlab.aggregate.COUNT()
        }
    ).sort('count', ascending=False)

In [24]:
most_recommended_songs.head()

song,count
Undo - Björk,431
Secrets - OneRepublic,374
Revelry - Kings Of Leon,236
You're The One - DwightYoakam ...,162
Sehr kosmisch - Harmonia,126
Fireflies - CharttraxxKaraoke ...,109
Hey_ Soul Sister - Train,98
Horn Concerto No. 4 in Eflat K495: II. Romance ...,91
OMG - Usher featuringwill.i.am ...,58
The Scientist - Coldplay,47
